In [2]:
import pyprind
import pandas as pd
import os

In [3]:
basepath = "/users/zabidi/Documents/IMDB_dateset/aclImdb/"

In [13]:
labels = {'pos':1,
         'neg':0}
pbar = pyprind.ProgBar(5000)
df = pd.DataFrame()

for s in ('test','train'):
    for l in ('pos','neg'):
        
        path = basepath + s + "/" + l
#         path = os.path.join(basepath, s, l)
        
#         print(path)
        
        for file in sorted(os.listdir(path)):
            with open(path + "/" + file, 'r',
                      encoding = 'utf-8') as infile:
                txt = infile.read()
            df = df.append([[txt, labels[l]]], ignore_index = True)
            pbar.update()
            
df.columns = ['review', 'sentiment']

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:09


In [15]:
import numpy as np
np.random.seed(0)
#shuffle data
df = df.reindex(np.random.permutation(df.index))
df.to_csv('movie_data.csv', index = False, encoding = 'utf-8')

In [16]:
df = pd.read_csv('movie_data.csv', encoding='utf-8')
df.head(3)

,review,sentiment
0,"Election is a Chinese mob movie, or triads in ...",1
1,I was just watching a Forensic Files marathon ...,0
2,Police Story is a stunning series of set piece...,1


## Transform words into feature vectors

In [21]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer()
docs = np.array(['The sun is shining',
                'The weather is sweet',
                'The sun is shining, the weather is sweet'
                 'and one and one is two'])
bag = count.fit_transform(docs)
print(bag.toarray())

[[0 1 0 1 1 0 0 1 0 0]
 [0 1 0 0 0 1 0 1 0 1]
 [1 3 2 1 1 0 1 2 1 1]]


In [22]:
print(count.vocabulary_)

{'the': 7, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 9, 'sweet': 5, 'sweetand': 6, 'one': 2, 'and': 0, 'two': 8}


In [24]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf = TfidfTransformer(use_idf = True,
                        norm = 'l2',
                        smooth_idf = True)
np.set_printoptions(precision = 2)
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[0.   0.43 0.   0.56 0.56 0.   0.   0.43 0.   0.  ]
 [0.   0.39 0.   0.   0.   0.66 0.   0.39 0.   0.5 ]
 [0.27 0.49 0.55 0.21 0.21 0.   0.27 0.32 0.27 0.21]]


In [25]:
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                       text)
    text = (re.sub('[\W]+', ' ', text.lower()) +
        ' '.join(emoticons).replace('-', ''))
    return text

In [29]:
df['review']=df['review'].apply(preprocessor)

In [30]:
def tokenizer(text):
    return text.split()

In [32]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [33]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop  = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/zabidi/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [35]:
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

In [37]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)
param_grid = [{'vect__ngram_range': [(1,1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer,tokenizer_porter],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              {'vect__ngram_range': [(1,1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer,
                     tokenizer_porter],
               'vect__use_idf':[False],
               'vect__norm':[None],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]}]

lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf',
                      LogisticRegression(random_state=0,
                                         solver='liblinear'))])
gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                           scoring='accuracy',
                           cv=5, verbose=2,
                           n_jobs=1)
gs_lr_tfidf.fit(X_train, y_train)

NameError: name 'Pipeline' is not defined